In [10]:
# To run this in Google Colab, uncomment the following line
# !pip install geometric_kernels

# If you want to use a version of thnew_space.dim = G.dim - H.dime library from a specific branch on GitHub,
# say, from the "devel" branch, uncomment the line below instead
# !pip install "git+https://github.com/GPflow/GeometricKernels@devel"

: 

In [11]:
# add the parent directory if you just cloned the repository
import sys
sys.path.append("..")

## Basics

In [12]:
# Import a backend, we use numpy in this example.
import numpy as np
import lab as B
# Import the geometric_kernels backend.
import geometric_kernels

# Note: if you are using a backend other than numpy,
# you _must_ uncomment one of the following lines
# import geometric_kernels.tensorflow
# import geometric_kernels.torch
# import geometric_kernels.jax

# Import a space and an appropriate kernel.
from geometric_kernels.kernels import MaternGeometricKernel
from geometric_kernels.spaces import Grassmannian, SpecialOrthogonal
from geometric_kernels.spaces.grassmannian import _SOxSO, _S_OxO
import matplotlib as mpl
import matplotlib.pyplot as plt

In [13]:
m, n = 3, 6

key = np.random.RandomState(1234)

grassmannian = Grassmannian(m=m, n=n)
so = SpecialOrthogonal(n=n)
stab = _S_OxO(n-m, m)

In [ ]:
kernel = MaternGeometricKernel(grassmannian)
kernel_so = MaternGeometricKernel(so)
eig_grass = kernel.eigenfunctions
eig_so = kernel_so.eigenfunctions

In [6]:
key, xs_grass = grassmannian.random(key, 5, project=True)
xs_so = grassmannian.embed_manifold(xs_grass)
key, xs_stab = stab.random(key, 100000)

xs_so_stab = np.einsum('nij,mjk->nmik', xs_so, xs_stab)

xs_grass_torus = eig_grass._torus_representative(xs_grass)
xs_so_stab_torus = eig_so._torus_representative(xs_so_stab)
xs_stab_torus = eig_so._torus_representative(xs_stab)

In [ ]:
from collections import defaultdict
eigs = defaultdict(dict)
for zsf, sgn, eigval in zip(kernel.eigenfunctions._zsf, kernel.eigenfunctions._signatures, kernel.eigenfunctions._eigenvalues):
    eigs[sgn]['zsf'] = zsf
    eigs[sgn]['eig_grass'] = eigval

for chi, sgn, eigval in zip(kernel_so.eigenfunctions._characters, kernel_so.eigenfunctions._signatures, kernel_so.eigenfunctions._eigenvalues):
    eigs[sgn]['chi'] = chi
    eigs[sgn]['eig_so'] = eigval

for sgn, eig in eigs.items():
    print(f"Signature: {sgn}")
    print(f"  Eigenvalue (Grassmannian): {eig.get('eig_grass')}")
    print(f"  Eigenvalue (SO): {eig.get('eig_so')}")

In [ ]:
for sgn, eig in eigs.items():
    zsf = eig.get('zsf')
    chi = eig.get('chi')
    if (chi is not None) and (zsf is not None):
        zsf_xs = zsf(xs_grass_torus)
        chi_xs_so_stab = chi(xs_so_stab_torus)
        chi_xs_stab = chi(xs_stab_torus)
        chi_xs_so_stab_avg = np.mean(chi_xs_so_stab, axis=0)
        chi_xs_stab_avg = np.mean(chi_xs_stab)
        print(sgn, ': ', chi_xs_stab_avg.real)

Now we evaluate the two kernel matrices.

In [ ]:
kernel_mat_32  = kernel.K(params_32,  xs, xs)
kernel_mat_inf = kernel.K(params_inf, xs, xs)

In [ ]:
id_ = np.eye(n).reshape(1, n, n)
id_ = grassmannian.project_to_manifold(id_)
torus_repr = kernel.eigenfunctions._torus_representative(id_)
kernel.eigenfunctions._zsf[7](torus_repr)

In [ ]:
kernel.eigenfunctions.G_dimensions, kernel.eigenfunctions._eigenvalues

In [ ]:
kernel.eigenfunctions._addition_theorem(id_,id_)

In [ ]:
xs_ = xs[:, :m, :]
print(xs_.shape)
args_matrix = np.einsum('...ij, ...kj->...ik', xs_,xs_)
eigenvalues = np.linalg.eigvalsh(args_matrix)
eigenvalues.shape

In [ ]:
xs_diff_xs = kernel.eigenfunctions._difference(xs, xs)
xs_diff_xs[2][2]

In [ ]:
np.set_printoptions(precision=3, suppress=True)

xs_embed_1 = grassmannian.embed_manifold(xs)
xs_embed_2 = grassmannian.embed_manifold(xs)
diff = kernel.eigenfunctions.G_eigenfunctions._difference(xs_embed_1, xs_embed_2)
print(diff[0,0])
xs_1, xs_2 = xs_embed_1[0], xs_embed_2[0]
xs_2.T @ xs_1

In [ ]:
xs_embed_2_inv = B.transpose(xs_embed_2)
xs_embed_1_ = B.tile(xs_embed_1[..., None, :, :], 1, xs_embed_2.shape[0], 1, 1)  # (N, N2, n, n)
xs_embed_2_inv_ = B.tile(xs_embed_2_inv[None, ..., :, :], xs_embed_1.shape[0], 1, 1, 1)  # (N, N2, n, n)
print(xs_embed_1_.shape, xs_embed_2_inv_.shape)
diff = B.reshape(
    B.matmul(xs_embed_1_, xs_embed_2_inv_), xs_embed_1.shape[0], xs_embed_2.shape[0], xs_embed_1.shape[-1], xs_embed_1.shape[-1]
)  # (N, N2, n, n)
diff[0][0]

In [ ]:
xs_embed_1[0], xs_embed_2_inv[0]

In [ ]:
print(xs_embed_1[0],'\n', xs_embed_2_inv[0])
diff = np.einsum('ij,jk-> ik', xs_embed_2_inv[0], xs_embed_1[0])
diff

In [ ]:
B.matmul(xs_embed_1_, xs_embed_2_inv_)

In [ ]:
B.transpose(xs_embed_1)[0] @ xs_embed_2[0]

In [ ]:
xs_diff_xs = kernel.eigenfunctions._difference(xs, xs)
grassmannian.embed_manifold(xs)
print(xs_diff_xs[0,0,...])
kernel.eigenfunctions._torus_representative(xs_diff_xs[:1,:1,...])

In [ ]:
grassmannian.embed_manifold(xs)[0], xs[0]

In [37]:
xs_torus = kernel.eigenfunctions._torus_representative(xs)

In [ ]:
xs.shape, g.shape

In [ ]:
N = 10000
zsfs = kernel.eigenfunctions._zsf
key, g = grassmannian.random(key, N, project=False)
g_xs = np.einsum('nij,mjk->nmik', g, xs)
g_torus = kernel.eigenfunctions._torus_representative(grassmannian.project_to_manifold(g))
g_xs_torus = kernel.eigenfunctions._torus_representative(g_xs)
for zsf in zsfs:
    zsf_g_xs = zsf(g_xs_torus)
    zsf_g = zsf(g_torus)
    zsf_xs_alt = np.einsum('in,i->n', zsf_g_xs, zsf_g)/N
    zsf_xs = zsf(xs_torus)
    print(zsf_xs_alt/zsf_xs)


In [ ]:
xs_embed = grassmannian.embed_manifold(xs)
diff = kernel.eigenfunctions.G_eigenfunctions._difference(xs_embed, xs_embed)
diff[0][2]

In [ ]:
torus_repr = kernel.eigenfunctions._torus_representative(diff)
torus_repr


Finally, we visualize these matrices using `imshow`.

In [ ]:
# find common range of values
minmin = np.min([np.min(kernel_mat_32), np.min(kernel_mat_inf)])
maxmax = np.max([np.max(kernel_mat_32), np.max(kernel_mat_inf)])

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
cmap = plt.get_cmap('viridis')

ax1.imshow(kernel_mat_32, vmin=minmin, vmax=maxmax, cmap=cmap)
ax1.set_title('k_32')
ax1.set_axis_off()

ax2.imshow(kernel_mat_inf, vmin=minmin, vmax=maxmax, cmap=cmap)
ax2.set_title('k_inf')
ax2.set_axis_off()

# add space for color bar
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.25, 0.02, 0.5])

# add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap,
                           norm=plt.Normalize(vmin=minmin, vmax=maxmax))
fig.colorbar(sm, cax=cbar_ax)

plt.show()

### Visualize Kernels

It is hard to visualize functions on $\mathrm{Gr}(m, n)$.
Functions are only easy to visualize on domains of dimension not higher than $2$.
However, we have $\dim \left(\mathrm{Gr}(m, n)\right) = m(n-m)$ which is greater than $2$ in almost all cases.

In [Stiefel.ipynb](./Stiefel.ipynb), we visualize functions on the Stiefel manifold $\mathrm{V}(m, n)$. Since all elements of the Grassmannian $\mathrm{Gr}(m, n)$ are equivalence classes of elements of $\mathrm{V}(m, n)$, we can use the same visualization trick as in [Stiefel.ipynb](./Stiefel.ipynb). It is important to be mindful of the fact that we might stumble upon elements of $\mathrm{V}(m, n)$ that represent the same element of $\mathrm{Gr}(m, n)$.

In [29]:
# define discretization
_NUM_ANGLES = 128

key, U = grassmannian.G.random(key, 2)  # stiefel.G returns an object representing SO(n)
U1 = U[0, :]
U2 = U[1, :]

# generate a grid on [0, 2 \pi)
angles = np.linspace(0, 2*np.pi, num=_NUM_ANGLES)
embedding = np.broadcast_to(np.eye(n), (_NUM_ANGLES, n, n)).copy()
embedding[:, 0, 0] = np.cos(angles)
embedding[:, 0, 1] = -np.sin(angles)
embedding[:, 1, 0] = np.sin(angles)
embedding[:, 1, 1] = np.cos(angles)

base_point = grassmannian.project_to_manifold(embedding[[0], :, :])

def conj_batch(U, emb):
    """ Compute U^T * emb[i, :] * U for all i.
    :param U:   [n, n] array
    :param emb: [N, n, n] array
    
    :return:    [N, n, n] array
    """
    return np.tensordot(np.tensordot(emb, U, axes=([2], [0])),
                        U.T, axes=([1], [1])
                       )

other_points1 = grassmannian.project_to_manifold(conj_batch(U1, embedding))
other_points2 = grassmannian.project_to_manifold(conj_batch(U2, embedding))

The next cell evaluates $k_{\nu, \kappa}($ `base_point` $, x)$ for $x \in $ `other_points1` and $x \in $ `other_points2`, for $\nu$ either $3/2$ or $\infty$.

In [ ]:
kernel_vals_32_1  = kernel.K(params_32,  base_point, other_points1)
kernel_vals_32_2  = kernel.K(params_32,  base_point, other_points2)
kernel_vals_inf_1 = kernel.K(params_inf, base_point, other_points1)
kernel_vals_inf_2 = kernel.K(params_inf, base_point, other_points2)

Finally, we are ready to plot the results.

In [ ]:
x_circle = np.cos(angles)
y_circle = np.sin(angles)
z_circle = np.zeros_like(angles) # z=0 for the circle

# Red ball position
red_ball_position = (x_circle[0], y_circle[0], z_circle[0])


def plot_kernel(ax, values, title):
    # Draw the unit circle
    ax.plot(x_circle, y_circle, z_circle, linestyle='dashed', color='gray')
    # Draw the red ball
    ax.scatter(*red_ball_position, color="r", s=50)
    # Draw the vertical dashed line
    ax.plot([1, 1], [0, 0], np.linspace(0, values[0, 0], 2), linestyle='dashed', color='gray')
    # Plot the new function evaluated only on the unit circle
    ax.plot(x_circle, y_circle, values[0, :], color='b')
    # Set the viewing angle for better visualization
    ax.view_init(elev=20., azim=180+30)
    ax.set_title(title)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(12.8, 9.6), nrows=2, ncols=2,
                               subplot_kw=dict(projection='3d',
                                               computed_zorder=False))

plot_kernel(ax1, kernel_vals_32_1,  r'$k_{3/2, \kappa}($base_point, other_points1$)$')
plot_kernel(ax2, kernel_vals_32_2,  r'$k_{3/2, \kappa}($base_point, other_points2$)$')
plot_kernel(ax3, kernel_vals_inf_1, r'$k_{\infty, \kappa}($base_point, other_points1$)$')
plot_kernel(ax4, kernel_vals_inf_2, r'$k_{\infty, \kappa}($base_point, other_points2$)$')


# Display the plot
plt.show()

## Feature Maps and Sampling

Here we show how to get an approximate finite-dimensional feature map for heat and Matérn kernels on the sphere, i.e. such $\phi$ that
$$
k(x, x') \approx \langle \phi(x), \phi(x') \rangle_{\mathbb{R}^M}.
$$
This might be useful for speeding up computations.
We showcase this below by showing how to efficiently sample the Gaussian process $\mathrm{GP}(0, k)$.

For a brief theoretical introduction into feature maps, see this [documentation page](https://gpflow.github.io/GeometricKernels/theory/feature_maps.html).

### Defining a Feature Map

The simplest way to get an approximate finite-dimensional feature map is to use the `default_feature_map` function from `geometric_kernels.kernels`.
It has an optional keyword argument `num` which determines the number of features, the $M$ above.
Below we rely on the default value of `num`.

In [32]:
from geometric_kernels.kernels import default_feature_map

feature_map = default_feature_map(kernel=kernel)

The resulting `feature_map` is a function that takes the array of inputs and parameters of the kernel.
There is also an optional parameter `normalize` that determines if $\langle \phi(x), \phi(x) \rangle_{\mathbb{R}^M} \approx 1$ or not.
For the (hyper)sphere, `normalize` follows the standard behavior of `MaternKarhunenLoeveKernel`, being `True` by default.

`feature_map` outputs a tuple.
Its **second** element is $\phi(x)$ evaluated at all inputs $x$.
Its first element is either `None` for determinstic feature maps, or contains the updated `key` for randomized feature maps which take `key` as a keyword argument.
For `default_feature_map` on a `Grassmannian` space, the first element is `key` since the feature map is *random*.

In the next cell, we evaluate the feature map at random points, using `params_32` as kernel parameters.
We check the basic property of the feature map: $k(x, x') \approx \langle \phi(x), \phi(x') \rangle_{\mathbb{R}^M}$.

In [ ]:
# xs are random points from above
key, embedding = feature_map(xs, params_32, key=key)

print('xs (shape = %s):\n%s' % (xs.shape, xs))
print('')
print('emedding (shape = %s):\n%s' % (embedding.shape, embedding))

kernel_mat_32  = kernel.K(params_32,  xs, xs)
kernel_mat_32_alt = np.matmul(embedding, embedding.T)

print('')
print('||k(xs, xs) - phi(xs) * phi(xs)^T||/||k(xs, xs)|| =', np.linalg.norm(kernel_mat_32 - kernel_mat_32_alt)/np.linalg.norm(kernel_mat_32))

**Unfortunately:** $k(x, x') \approx \langle \phi(x), \phi(x') \rangle_{\mathbb{R}^M}$ does not hold, which is either caused by a bug or by the insufficiency of the default approximation order in this case. **TODO:** investigate this.

### Efficient Sampling using Feature Maps

GeometricKernels provides a simple tool to efficiently sample (without incurring cubic costs) the Gaussian process $f \sim \mathrm{GP}(0, k)$, based on an approximate finite-dimensional feature map $\phi$.
The underlying machinery is briefly discussed in this [documentation page](https://gpflow.github.io/GeometricKernels/theory/feature_maps.html).

The function `sampler` from `geometric_kernels.sampling` takes in a feature map and, optionally, the keyword argument `s` that specifies the number of samples to generate.
It returns a function we name `sample_paths`.
Since we are going to compute each of the two samples at two different sets of inputs, `other_points1` and `other_points2`, we make sure the randomness if fixed by using the `make_deterministic` function.

`sample_paths` operates much like `feature_map` above: it takes in the points where to evaluate the samples and kernel parameters.
Additionally, it takes in the keyword argument `key` that specifies randomness in the JAX style.
**However**, in our specific case, this keyword argument is not needed as it is automatically supplied by the `make_deterministic` wrapper.
`sample_paths` returns a tuple.
Its first element is the updated `key`.
Its second element is an array containing the value of samples evaluated at the input points.

In [ ]:
from geometric_kernels.sampling import sampler
from geometric_kernels.utils.utils import make_deterministic

# introduce random state for reproducibility (optional)
# `key` is jax's terminology
key = np.random.RandomState(seed=1234)

sample_paths = make_deterministic(sampler(feature_map, s=2), key)

# new random state is returned along with the samples
key, samples = sample_paths(xs, params_32)

print('Two samples evaluated at the xs are:')
print(samples)

#### Visualizing Samples
Here we visualize samples as functions on the sphere.

In [ ]:
key, samples_other_points1 = sample_paths(other_points1, params_32)
key, samples_other_points2 = sample_paths(other_points2, params_32)

sample1_other_points1 = samples_other_points1[:, 0]
sample2_other_points1 = samples_other_points1[:, 1]
sample1_other_points2 = samples_other_points2[:, 0]
sample2_other_points2 = samples_other_points2[:, 1]

x_circle = np.cos(angles)
y_circle = np.sin(angles)
z_circle = np.zeros_like(angles) # z=0 for the circle

def plot_sample(ax, values, title):
    # Draw the unit circle
    ax.plot(x_circle, y_circle, z_circle, linestyle='dashed', color='gray')
    # Plot the new function evaluated only on the unit circle
    ax.plot(x_circle, y_circle, values, color='b')
    # Set the viewing angle for better visualization
    ax.view_init(elev=20., azim=180+30)
    ax.set_title(title)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(12.8, 9.6), nrows=2, ncols=2,
                               subplot_kw=dict(projection='3d',
                                               computed_zorder=False))

plot_sample(ax1, samples_other_points1[:, 0], r'Sample #1: $f(\cdot)$ for $\cdot$ in other_points1, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')
plot_sample(ax2, samples_other_points2[:, 0], r'Sample #1: $f(\cdot)$ for $\cdot$ in other_points2, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')
plot_sample(ax3, samples_other_points1[:, 1], r'Sample #2: $f(\cdot)$ for $\cdot$ in other_points1, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')
plot_sample(ax4, samples_other_points2[:, 1], r'Sample #2: $f(\cdot)$ for $\cdot$ in other_points2, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')


# Display the plot
plt.show()

## The Simplest Special Case: Real Projective Space $\mathrm{RP}^3$

Here we visualize the kernel and samples in the simplest special case, for the real projective space $\mathrm{RP}^3$ that can be thought of as the standard two-dimensional sphere for which every pair of opposite points, i.e. $x$ and $-x$, is identified. In this case, we can visualize functions on $\mathrm{RP}^3$ as functions on the $2$-dimensional sphere.

In [ ]:
key = np.random.RandomState(1234)

projective = Grassmannian(m=1, n=3, key=key)

kernel_projective = MaternGeometricKernel(projective)

params_32  = {"lengthscale": np.array([0.125]), "nu": np.array([3/2])}
params_inf = {"lengthscale": np.array([0.125]), "nu": np.array([np.inf])}

### Visualize Kernels
We use the same code as in [Hypersphere.ipynb](./Hypersphere.ipynb).

In [ ]:
base_point = np.array([[0.0, 0.0, 1.0]])

# define latitude and longitude discretization
_NUM_LATS = 128
_NUM_LONGS = 128

# generate a grid on the sphere
lats, longs =  np.mgrid[0:2*np.pi:_NUM_LATS*1j, 0:np.pi:_NUM_LONGS*1j]

other_points_xs = np.sin(longs) * np.cos(lats)
other_points_ys = np.sin(longs) * np.sin(lats)
other_points_zs = np.cos(longs)

other_points = np.c_[np.ravel(other_points_xs),
                     np.ravel(other_points_ys),
                     np.ravel(other_points_zs)]

# The elements of `projective` are matrices 3x1 rather than 3-dim vectors.
base_point = base_point[..., np.newaxis]
other_points = other_points[..., np.newaxis]


kernel_vals_32  = kernel_projective.K(params_32,  base_point, other_points)
kernel_vals_inf = kernel_projective.K(params_inf, base_point, other_points)

fig, (ax1, ax2) = plt.subplots(figsize=(12.8, 4.8), nrows=1, ncols=2,
                               subplot_kw=dict(projection='3d',
                                               computed_zorder=False))

cmap = plt.get_cmap('viridis')

surf1 = ax1.plot_surface(other_points_xs, other_points_ys, other_points_zs,
                         facecolors=cmap(kernel_vals_32[0, :].reshape(_NUM_LATS, _NUM_LONGS)),  cstride=1, rstride=1)
surf2 = ax2.plot_surface(other_points_xs, other_points_ys, other_points_zs,
                         facecolors=cmap(kernel_vals_inf[0, :].reshape(_NUM_LATS, _NUM_LONGS)), cstride=1, rstride=1)

# Remove axis
ax1._axis3don = False
ax2._axis3don = False

# Set aspect ratio
ax1.set_box_aspect((np.ptp(other_points_xs), np.ptp(other_points_ys),
                    np.ptp(other_points_zs)), zoom=1.5)
ax2.set_box_aspect((np.ptp(other_points_xs), np.ptp(other_points_ys),
                    np.ptp(other_points_zs)), zoom=1.5)

# plot the base point
ax1.scatter(base_point[0, 0], base_point[0, 1], base_point[0, 2], s=10, c='r', marker='D')
ax2.scatter(base_point[0, 0], base_point[0, 1], base_point[0, 2], s=10, c='r', marker='D')

# find common range of values
minmin_vis = np.min([np.min(kernel_vals_32), np.min(kernel_vals_inf)])
maxmax_vis = np.max([np.max(kernel_vals_32), np.max(kernel_vals_inf)])

ax1.set_title(r'$k_{3/2, \kappa}($base_point, other_points$)$')
ax2.set_title('$k_{\infty, \kappa}($base_point, other_points$)$')

# add color bar
sm = plt.cm.ScalarMappable(cmap=cmap,
                           norm=plt.Normalize(vmin=minmin_vis, vmax=maxmax_vis))
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.25, 0.02, 0.5])
fig.colorbar(sm, cax=cbar_ax)

plt.show()

### Visualize Samples
We use the same code as in [Hypersphere.ipynb](./Hypersphere.ipynb).

In [ ]:
from geometric_kernels.kernels import default_feature_map
from geometric_kernels.sampling import sampler

feature_map = default_feature_map(kernel=kernel_projective)
sample_paths = sampler(feature_map, s=2)

key = np.random.RandomState(seed=1234)
key, samples = sample_paths(other_points, params_32, key=key)

sample1 = samples[:, 0]
sample2 = samples[:, 1]

fig, (ax1, ax2) = plt.subplots(figsize=(12.8, 4.8), nrows=1, ncols=2,
                               subplot_kw=dict(projection='3d',
                                               computed_zorder=False))

cmap = plt.get_cmap('viridis')

surf1 = ax1.plot_surface(other_points_xs, other_points_ys, other_points_zs,
                         facecolors=cmap(sample1.reshape(_NUM_LATS, _NUM_LONGS)),  cstride=1, rstride=1)
surf2 = ax2.plot_surface(other_points_xs, other_points_ys, other_points_zs,
                         facecolors=cmap(sample2.reshape(_NUM_LATS, _NUM_LONGS)), cstride=1, rstride=1)

# Remove axis
ax1._axis3don = False
ax2._axis3don = False

# Set aspect ratio
ax1.set_box_aspect((np.ptp(other_points_xs), np.ptp(other_points_ys),
                    np.ptp(other_points_zs)), zoom=1.5)
ax2.set_box_aspect((np.ptp(other_points_xs), np.ptp(other_points_ys),
                    np.ptp(other_points_zs)), zoom=1.5)

# find common range of values
minmin_vis = np.min([np.min(sample1), np.min(sample2)])
maxmax_vis = np.max([np.max(sample1), np.max(sample2)])

ax1.set_title('Sample #1: $f(\cdot)$ for $\cdot$ in other_points, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')
ax2.set_title('Sample #2: $f(\cdot)$ for $\cdot$ in other_points, $f \sim \mathrm{GP}(0, k_{3/2, \kappa})$')

# add color bar
sm = plt.cm.ScalarMappable(cmap=cmap,
                           norm=plt.Normalize(vmin=minmin_vis, vmax=maxmax_vis))
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.25, 0.02, 0.5])
fig.colorbar(sm, cax=cbar_ax)
plt.show()

## Citation

If you are using Lie groups and GeometricKernels, please consider citing

```
@article{azangulov2022,
    title={Stationary kernels and Gaussian processes on Lie groups and their homogeneous spaces I: the compact case},
    author={Azangulov, Iskander and Smolensky, Andrei and Terenin, Alexander and Borovitskiy, Viacheslav},
    journal={arXiv preprint arXiv:2208.14960},
    year={2022}
}
```